# Practice on Keras Functional API

In [5]:
# imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
# for types hints
from typing import Tuple, Callable, List
from tensorflow import Tensor

In [6]:
# check for gpus
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
# list their names
tf.config.experimental.list_physical_devices('GPU')

Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
# mnist dataset
def load_and_preprocess_mnist() -> Tuple[Tuple[Tensor, Tensor], Tuple[Tensor, Tensor]]:
    """
    Load the MNIST dataset, preprocess images, and perform one-hot encoding of labels.

    :return: Tuple of training data (x_train, y_train) and testing data (x_test, y_test).
    """
    # Load MNIST dataset
    (x_train, y_train), (x_test, y_test) =  tf.keras.datasets.mnist.load_data()

    # Reshape and normalize images
    x_train = x_train.reshape(60000, 784).astype('float32') / 255
    x_test = x_test.reshape(10000, 784).astype('float32') / 255

    # One-hot encoding of labels
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)

    return (x_train, y_train), (x_test, y_test)

In [8]:

(x_train, y_train), (x_test, y_test) = load_and_preprocess_mnist()


In [74]:
# building the model
def build_model() -> tf.keras.Model:
    """
    Build a simple MLP model for MNIST classification.
    :return: A tf.keras Model with inputs and outputs defined.
    """
    inputs = tf.keras.Input(shape=(784,), name='input')  # input layer
    x = tf.keras.layers.Dense(64, activation='relu', name='hidden1')(inputs)  # hidden layer
    x = tf.keras.layers.Dense(64, activation='relu', name='hidden2')(x)  # hidden layer
    outputs = tf.keras.layers.Dense(10, activation='softmax', name='output')(x)  # output layer
    
    return tf.keras.Model(inputs=inputs, outputs=outputs)

model_ce = build_model()
model_fl = build_model()
model_rl = build_model()

In [10]:
optimizer = tf.keras.optimizers.SGD()
ce_loss = tf.keras.losses.CategoricalCrossentropy()
metrics = [tf.keras.metrics.CategoricalAccuracy()]
# compiling the model
model_ce.compile(optimizer=optimizer, loss=ce_loss, metrics=metrics)
# training the model
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
model_ce.fit(x_train, y_train, batch_size=64, epochs=100, validation_split=0.2, callbacks=[early_stopping_callback])
# evaluating the model
test_scores = model_ce.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1]*100)

# saving the model
path = './weights/mnist_ce_model.keras'
model_ce.save(path)

Epoch 1/100


2023-08-18 09:13:22.022654: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-08-18 09:13:22.026325: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f3775d875a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-18 09:13:22.026361: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-08-18 09:13:22.246566: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-18 09:13:22.257857: W tensorflow/compiler/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:543] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  /usr/lo

750/750 [==============================] - 8s 8ms/step - loss: 1.1604 - categorical_accuracy: 0.6918 - val_loss: 0.4948 - val_categorical_accuracy: 0.8758
Epoch 2/100
750/750 [==============================] - 6s 8ms/step - loss: 0.4346 - categorical_accuracy: 0.8825 - val_loss: 0.3555 - val_categorical_accuracy: 0.9009
Epoch 3/100
750/750 [==============================] - 6s 8ms/step - loss: 0.3533 - categorical_accuracy: 0.9003 - val_loss: 0.3089 - val_categorical_accuracy: 0.9120
Epoch 4/100
750/750 [==============================] - 6s 8ms/step - loss: 0.3163 - categorical_accuracy: 0.9099 - val_loss: 0.2836 - val_categorical_accuracy: 0.9208
Epoch 5/100
750/750 [==============================] - 5s 7ms/step - loss: 0.2911 - categorical_accuracy: 0.9175 - val_loss: 0.2653 - val_categorical_accuracy: 0.9245
Epoch 6/100
750/750 [==============================] - 6s 8ms/step - loss: 0.2716 - categorical_accuracy: 0.9232 - val_loss: 0.2494 - val_categorical_accuracy: 0.9306
Epoch 7/10

In [11]:
focal_loss = tf.keras.losses.CategoricalFocalCrossentropy()
# compiling the model
optimizer = tf.keras.optimizers.SGD()
model_fl.compile(optimizer=optimizer, loss=focal_loss, metrics=metrics)
# training the model
model_fl.fit(x_train, y_train, batch_size=64, epochs=100, validation_split=0.2, callbacks=[early_stopping_callback])
# evaluating the model
test_scores = model_fl.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1]*100)
# saving the model
path = './weights/mnist_focal_model.keras'
model_fl.save(path)

Epoch 1/100
750/750 [==============================] - 8s 9ms/step - loss: 0.3605 - categorical_accuracy: 0.5545 - val_loss: 0.2290 - val_categorical_accuracy: 0.7343
Epoch 2/100
750/750 [==============================] - 7s 10ms/step - loss: 0.1592 - categorical_accuracy: 0.7690 - val_loss: 0.1063 - val_categorical_accuracy: 0.8219
Epoch 3/100
750/750 [==============================] - 7s 9ms/step - loss: 0.0954 - categorical_accuracy: 0.8232 - val_loss: 0.0762 - val_categorical_accuracy: 0.8548
Epoch 4/100
750/750 [==============================] - 7s 9ms/step - loss: 0.0749 - categorical_accuracy: 0.8511 - val_loss: 0.0636 - val_categorical_accuracy: 0.8723
Epoch 5/100
750/750 [==============================] - 7s 9ms/step - loss: 0.0648 - categorical_accuracy: 0.8667 - val_loss: 0.0568 - val_categorical_accuracy: 0.8812
Epoch 6/100
750/750 [==============================] - 7s 10ms/step - loss: 0.0586 - categorical_accuracy: 0.8780 - val_loss: 0.0525 - val_categorical_accuracy: 0.8

In [92]:
# defining rational loss function
# RL(p_t) = 1/p_t * -p * log(p_t)
    # TODO: plot the function
def rational_loss(y_true, y_pred, reduction=tf.keras.losses.Reduction.AUTO):
    """
    Rational Loss for multi-class classification, tf.keras style.
    RL(p_t) = - 1/p_t * log(p_t), where p_t is the probability associated with the true class.

    :param y_true: Ground truth labels, shape of [batch_size, num_classes].
    :param y_pred: Predicted class probabilities, shape of [batch_size, num_classes].
    :param reduction: Reduction method to apply to the loss (default tf.keras.losses.Reduction.NONE).
    :return: A scalar representing the mean rational loss over the batch.
    NOTE: written assuming GPU support to make use of fast Tensor operations.
    """
    # Create a Categorical Cross-Entropy loss instance
    cce = tf.keras.losses.CategoricalCrossentropy(
        reduction=tf.keras.losses.Reduction.NONE # Keep unreduced loss tensor
    )
    
    # clip the predicted probabilities to avoid log(0)
    _y_pred = K.clip(y_pred, 1e-2, 1.0)  # batch_sizexnum_classes
    cross_entropy = cce(y_true, _y_pred) # batch_sizex1
    # find the probability associated with the true class
    _y_true = K.argmax(y_true, axis=1)
    # get the predicted probability of the true class
    _y_pred = K.sum(_y_pred * y_true, axis=1)
    # print(f'y_true: {_y_true}')
    # print(f'y_pred: {_y_pred}')
    # print(f'cross entropy: {cross_entropy}')
    # rational loss by dividing the cross entropy by the predicted probability of the true class
    _rational_loss = cross_entropy / _y_pred # Rational loss
    
    if reduction == tf.keras.losses.Reduction.AUTO:
        return K.mean(_rational_loss)
    elif reduction == tf.keras.losses.Reduction.SUM:
        return K.sum(_rational_loss)
    else: # No reduction
        return _rational_loss



In [93]:
# test rational loss
y_true = np.array([[0, 1, 0], [0, 0, 1]])
y_pred = np.array([[0.05, 0.95, 0], [0.1, 0.8, 0.1]])
# cross entropy value 
cce = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
print(f'cross entropy: {cce(y_true, y_pred)}')
print(f'rational loss: {rational_loss(y_true, y_pred, reduction=tf.keras.losses.Reduction.NONE)}')

# with auto
cce = tf.keras.losses.CategoricalCrossentropy()
print(f'cross entropy: {cce(y_true, y_pred)}')
print(f'rational loss: {rational_loss(y_true, y_pred)}')



cross entropy: [0.05129329 2.30258509]
rational loss: [ 0.06446697 23.02585093]
cross entropy: 1.176939193690798
rational loss: 11.545158951939026


In [14]:
# def generate_data() -> Tuple[np.ndarray, np.ndarray]:
#     """
#     Generates synthetic y_true and y_pred data.
#     :return: y_true and y_pred arrays.
#     """
#     num_samples = 100
#     num_classes = 3
#     y_true = np.eye(num_classes)[np.random.choice(num_classes, num_samples)]
#     y_pred = np.random.rand(num_samples, num_classes)
#     y_pred /= y_pred.sum(axis=1, keepdims=True)
#     return y_true, y_pred
# 
# def plot_rational_loss() -> None:
#     """
#     Plots the rational loss for the generated data.
#     """
#     y_true, y_pred = generate_data()
#     rational_loss_fixed = rational_loss()
#     losses: List[float] = [rational_loss_fixed(y_t.reshape(1, -1), y_p.reshape(1, -1)).numpy() for y_t, y_p in zip(y_true, y_pred)]
#     plt.plot(losses)
#     plt.title("Rational Loss for Multi-Class Classification")
#     plt.xlabel("Sample")
#     plt.ylabel("Loss")
#     plt.show()
# 
# plot_rational_loss()

In [99]:
# build th emodel
model_rl = build_model()
# compiling the model
model_rl.compile(optimizer= tf.keras.optimizers.SGD(), loss=rational_loss, metrics=metrics)
# training the model
history = model_rl.fit(x_train, y_train, batch_size=64, epochs=100, validation_split=0.2, callbacks=[early_stopping_callback])
# evaluating the model
test_scores = model_rl.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1]*100)
# saving the model
path = './weights/mnist_rational_model.keras'
model_rl.save(path)

Epoch 1/100
750/750 [==============================] - 5s 6ms/step - loss: 11.0329 - categorical_accuracy: 0.7006 - val_loss: 4.4918 - val_categorical_accuracy: 0.8025
Epoch 2/100
750/750 [==============================] - 5s 6ms/step - loss: 6.0201 - categorical_accuracy: 0.7681 - val_loss: 3.0958 - val_categorical_accuracy: 0.8654
Epoch 3/100
750/750 [==============================] - 5s 6ms/step - loss: 3.2584 - categorical_accuracy: 0.8621 - val_loss: 2.9498 - val_categorical_accuracy: 0.8735
Epoch 4/100
750/750 [==============================] - 5s 7ms/step - loss: 2.3749 - categorical_accuracy: 0.8908 - val_loss: 2.2055 - val_categorical_accuracy: 0.9108
Epoch 5/100
750/750 [==============================] - 5s 6ms/step - loss: 2.3169 - categorical_accuracy: 0.8987 - val_loss: 2.2919 - val_categorical_accuracy: 0.8898
Epoch 6/100
750/750 [==============================] - 5s 6ms/step - loss: 2.1993 - categorical_accuracy: 0.9028 - val_loss: 2.5650 - val_categorical_accuracy: 0.88

In [57]:
# rebuilding the models
model_ce = build_model()
model_fl = build_model()
model_rl = build_model()


In [17]:
# imbalance 
def create_imbalanced_data(x, y, imbalance_rate=0.5):
    """
    Create an imbalanced dataset based on a given probability distribution.
    The probability for class d is given by: P(d) = 0.5^d / 2*(1 - 0.5^10)

    :param x: Features, shape of [total_samples, feature_dim].
    :param y: One-hot encoded labels, shape of [total_samples, num_classes].
    :param imbalance_rate: Base rate for the exponential decay of class frequency (default 0.5).
    :return: Tuple of imbalanced features and labels, shapes of [selected_samples, feature_dim] and [selected_samples, num_classes].
    """
    total_samples = len(y)
    a = imbalance_rate
    normalization_factor = 2 * (1 - a**10)

    indices_by_class = [np.where(y[:, d] == 1)[0] for d in range(10)]
    selected_indices = []

    for d in range(10):
        probability_d = (a**d) / normalization_factor
        frequency = int(total_samples * probability_d)
        np.random.shuffle(indices_by_class[d]) # Shuffle to ensure random selection
        selected_indices.extend(indices_by_class[d][:frequency])

    return x[selected_indices], y[selected_indices]


x_train_imbalanced, y_train_imbalanced = create_imbalanced_data(x_train, y_train)


In [95]:
# compiling the model
model_ce.compile(optimizer=tf.keras.optimizers.SGD(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=metrics)
model_fl.compile(optimizer=tf.keras.optimizers.SGD(), loss=tf.keras.losses.CategoricalFocalCrossentropy(), metrics=metrics)
model_rl.compile(optimizer=tf.keras.optimizers.SGD(), loss=rational_loss, metrics=metrics)

In [96]:
print("Training on imbalanced data:")
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
print("Cross Entropy:")
model_ce.fit(x_train_imbalanced, y_train_imbalanced, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stopping_callback])
print("Focal Loss:")
model_fl.fit(x_train_imbalanced, y_train_imbalanced, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stopping_callback])
print("Rational Loss:")
model_rl.fit(x_train_imbalanced, y_train_imbalanced, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stopping_callback])

Training on imbalanced data:
Cross Entropy:
Epoch 1/100
326/326 [==============================] - 3s 8ms/step - loss: 0.5654 - categorical_accuracy: 0.8745 - val_loss: 4.6957 - val_categorical_accuracy: 0.2501
Epoch 2/100
326/326 [==============================] - 3s 9ms/step - loss: 0.1445 - categorical_accuracy: 0.9619 - val_loss: 5.4116 - val_categorical_accuracy: 0.2649
Epoch 3/100
326/326 [==============================] - 3s 9ms/step - loss: 0.1118 - categorical_accuracy: 0.9673 - val_loss: 5.8288 - val_categorical_accuracy: 0.2622
Epoch 4/100
326/326 [==============================] - 3s 9ms/step - loss: 0.0976 - categorical_accuracy: 0.9721 - val_loss: 6.1346 - val_categorical_accuracy: 0.2660
Epoch 5/100
326/326 [==============================] - 3s 9ms/step - loss: 0.0894 - categorical_accuracy: 0.9737 - val_loss: 6.3106 - val_categorical_accuracy: 0.2670
Epoch 6/100
326/326 [==============================] - 3s 9ms/step - loss: 0.0834 - categorical_accuracy: 0.9753 - val_lo

In [97]:
# Balanced data
print("test on balanced set:")
ce_res = model_ce.evaluate(x_test, y_test)
fl_res = model_fl.evaluate(x_test, y_test)
rl_res = model_rl.evaluate(x_test, y_test)
print(f'ce: test loss {ce_res[0]}, test accuracy {ce_res[1]*100}')
print(f'fl: test loss {fl_res[0]}, test accuracy {fl_res[1]*100}')
print(f'rl: test loss {rl_res[0]}, test accuracy {rl_res[1]*100}')
# Imbalanced data
print("test on imbalanced data:")
x_test_imbalanced, y_test_imbalanced = create_imbalanced_data(x_test, y_test)
ce_ires = model_ce.evaluate(x_test_imbalanced, y_test_imbalanced)
fl_ires = model_fl.evaluate(x_test_imbalanced, y_test_imbalanced)
rl_ires = model_rl.evaluate(x_test_imbalanced, y_test_imbalanced)
print(f'ce: test loss {ce_ires[0]}, test accuracy {ce_ires[1]*100}')
print(f'fl: test loss {fl_ires[0]}, test accuracy {fl_ires[1]*100}')
print(f'rl: test loss {rl_ires[0]}, test accuracy {rl_ires[1]*100}')

test on balanced set:
313/313 [==============================] - 1s 4ms/step - loss: 34.6184 - categorical_accuracy: 0.4998
ce: test loss 4.038527488708496, test accuracy 39.59000110626221
fl: test loss 0.5938497185707092, test accuracy 30.000001192092896
rl: test loss 34.618350982666016, test accuracy 49.97999966144562
test on imbalanced data:
138/138 [==============================] - 0s 3ms/step - loss: 7.2159 - categorical_accuracy: 0.8791
ce: test loss 1.079444408416748, test accuracy 82.34891891479492
fl: test loss 0.21811704337596893, test accuracy 68.41505169868469
rl: test loss 7.215880393981934, test accuracy 87.9133403301239


In [ ]:
def accuracy_by_bins(model, x, y):
    """
    Calculate and print the accuracy of the given model for specific bins of classes.
    The bins are defined as: 0-1, 2-7, 8-9.

    :param model: Trained tf.keras model to evaluate.
    :param x: Input features, shape of [num_samples, feature_dim].
    :param y: One-hot encoded labels, shape of [num_samples, num_classes].
    """
    predictions = model.predict(x).argmax(axis=-1)
    true_labels = y.argmax(axis=-1)
    bins = [(0, 1), (2, 7), (8, 9)]
    for bin_start, bin_end in bins:
        mask = (true_labels >= bin_start) & (true_labels <= bin_end) 
        bin_accuracy = np.mean(predictions[mask] == true_labels[mask])
        print(f"Accuracy for bin {bin_start}-{bin_end}: {bin_accuracy*100}")

In [98]:
print("Accuracy by bins for balanced data:")
print("Cross Entropy:")
accuracy_by_bins(model_ce, x_test, y_test)
print("Focal Loss:")
accuracy_by_bins(model_fl, x_test, y_test)
print("Rational Loss:")
accuracy_by_bins(model_rl, x_test, y_test)

Accuracy by bins for balanced data:
Cross Entropy:
313/313 [==============================] - 1s 2ms/step
Accuracy for bin 0-1: 99.05437352245863
Accuracy for bin 2-7: 31.58251440189766
Accuracy for bin 8-9: 0.0
Focal Loss:
313/313 [==============================] - 1s 2ms/step
Accuracy for bin 0-1: 97.06855791962174
Accuracy for bin 2-7: 16.045408336157234
Accuracy for bin 8-9: 0.0
Rational Loss:
313/313 [==============================] - 1s 2ms/step
Accuracy for bin 0-1: 99.2434988179669
Accuracy for bin 2-7: 44.391731616401216
Accuracy for bin 8-9: 14.069591527987896


In [100]:
print("Accuracy by bins for imbalanced data:")
print("Cross Entropy:")
accuracy_by_bins(model_ce, x_test_imbalanced, y_test_imbalanced)
print("Focal Loss:")
accuracy_by_bins(model_fl, x_test_imbalanced, y_test_imbalanced)
print("Rational Loss:")
accuracy_by_bins(model_rl, x_test_imbalanced, y_test_imbalanced)

Accuracy by bins for imbalanced data:
Cross Entropy:
138/138 [==============================] - 1s 5ms/step
Accuracy for bin 0-1: 99.05437352245863
Accuracy for bin 2-7: 67.61819803746654
Accuracy for bin 8-9: 0.0
Focal Loss:
138/138 [==============================] - 1s 4ms/step
Accuracy for bin 0-1: 97.06855791962174
Accuracy for bin 2-7: 42.23907225691347
Accuracy for bin 8-9: 0.0
Rational Loss:
138/138 [==============================] - 1s 4ms/step
Accuracy for bin 0-1: 97.16312056737588
Accuracy for bin 2-7: 91.21320249776986
Accuracy for bin 8-9: 85.71428571428571
